# Poyang Lake Study Area

In [1]:
# load modules
import os
import numpy as np
import pickle
import warnings
import geopandas as gpd
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects


In [3]:
# region of interest (longitude, latitude in WGS84)
# can also be loaded from a .kml polygon
# kml_polygon = os.path.join(os.getcwd(), 'examples', 'PoyangLakeRiverMouth.kml')
# polygon = SDS_tools.polygon_from_kml(kml_polygon)

aoi_poly = gpd.read_file("area_crop2.geojson")
g = [i for i in aoi_poly.geometry]
x,y = g[0].exterior.coords.xy
coords = np.dstack((x,y)).tolist()
polygon = coords 
print(polygon)

# date range
dates = ['1985-01-01', '1985-08-01']

# satellite missions
sat_list = ['L5', 'L7', 'L8','S2']

# name of the site
sitename = 'POYANG2'

# filepath where data will be stored
filepath_data = os.path.join(os.getcwd(), 'data')

# put all the inputs into a dictionnary
inputs = {
    'polygon': polygon,
    'dates': dates,
    'sat_list': sat_list,
    'sitename': sitename,
    'filepath': filepath_data
        }


[[[116.1096442706144, 29.22680704226312], [116.1110162724817, 29.2109306394173], [116.1748499964451, 29.21393728784619], [116.1736960079658, 29.21975529322346], [116.1529766222694, 29.22375921247242], [116.1382100346204, 29.22595484190937], [116.1276922178228, 29.22743855292531], [116.1096442706144, 29.22680704226312]]]


In [3]:
## Check if the path exists or not
path_to_check   = os.path.join(filepath_data, sitename)

if os.path.exists(path_to_check) == False: 
    # before downloading the images, check how many images are available for your inputs
    SDS_download.check_images_available(inputs);

    #%% 2. Retrieve images

    # only uncomment this line if you want Landsat Tier 2 images (not suitable for time-series analysis)
    # inputs['include_T2'] = True

    # retrieve satellite images from GEE    
    metadata = SDS_download.retrieve_images(inputs)
else:
    print("You have downloaded files in the your directory! Let's reuse them!") 
    pass 

You have downloaded files in the your directory! Let's reuse them!


In [4]:
metadata = SDS_download.get_metadata(inputs) 

In [4]:
settings = { 
    # Temp flag for dirty hack 
    'dirty_hack_flag': True,
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': 32650,       # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': False,    # if True, shows each shoreline detection to the user for validation
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    'color_style': False,       # if True, saves figure as true color image. If False, saves figure as false color image. 
    # add the inputs defined previously
    'inputs': inputs,
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 9000,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
}

In [6]:
# save jpeg of processed satellite image
#SDS_preprocess.save_jpg(metadata, settings)

In [2]:
poy1 = gpd.read_file("poyang_refline1.geojson")
g = [i for i in poy1.geometry]
x,y = g[0].coords.xy
coords1 = np.dstack((x,y))[0]

poy2 = gpd.read_file("poyang_refline2.geojson")
print(poy2.crs)
g = [i for i in poy2.geometry]
x,y = g[0].coords.xy
coords2 = np.dstack((x,y))[0]


{'init': 'epsg:32650'}
{'init': 'epsg:32650'}
[[ 412149.80034375 3232131.01650064]
 [ 412189.53492618 3232301.57645421]
 [ 412245.86682956 3232397.5431283 ]
 [ 412306.08746773 3232498.48594656]
 [ 412385.3872062  3232569.93810793]
 [ 412452.499657   3232637.16027861]
 [ 412526.59365446 3232717.62323306]
 [ 412614.98211598 3232814.42149225]
 [ 412728.26712727 3232872.07177296]
 [ 412830.6480132  3232927.64458296]
 [ 412910.18417763 3232971.64077694]
 [ 413044.08233759 3233027.38413905]
 [ 413181.00525429 3233073.81748418]
 [ 413282.70116565 3233115.43806369]
 [ 413337.77366176 3233131.24656236]
 [ 413418.56837671 3233152.67075171]
 [ 413578.57616015 3233174.12299175]
 [ 413754.71555748 3233165.26899276]
 [ 413905.76307226 3233169.19453117]
 [ 413997.91703906 3233167.78574795]
 [ 414119.43892171 3233163.20077543]
 [ 414233.52642701 3233169.72606097]
 [ 414386.973317   3233153.48886457]
 [ 414509.92630365 3233150.95388483]
 [ 414592.15297234 3233129.61445335]
 [ 414691.07252597 3233104.23

In [8]:
'''if 'coords2' in globals():
    print("Using dual reference")
    refs = np.append(coords1, coords2, axis = 0)
elif 'coords2' in locals():
    print("Using dual reference")
    refs = np.append(coords1, coords2, axis = 0)
else:
    print("using single reference")
    refs = coords1
print(refs)'''


'if \'coords2\' in globals():\n    print("Using dual reference")\n    refs = np.append(coords1, coords2, axis = 0)\nelif \'coords2\' in locals():\n    print("Using dual reference")\n    refs = np.append(coords1, coords2, axis = 0)\nelse:\n    print("using single reference")\n    refs = coords1\nprint(refs)'

In [9]:
"""with open("data/POYANG2/POYANG2_reference_shoreline.pkl", "wb") as f:
    pickle.dump(refs, f)"""

'with open("data/POYANG2/POYANG2_reference_shoreline.pkl", "wb") as f:\n    pickle.dump(refs, f)'

In [10]:
%matplotlib qt
settings['reference_shoreline'] = SDS_preprocess.get_reference_sl(metadata, settings)
settings['max_dist_ref'] = 1000 # max distance (in meters) allowed from the reference shoreline

Reference shoreline already exists and was loaded


In [ ]:
#moment of truth
%matplotlib qt
output = SDS_shoreline.extract_shorelines(metadata, settings)

Mapping shorelines:
L5:   0%I am doing the dirty hack!
Could not map shoreline for this image: 1986-11-28-02-02-38_L5_POYANG2.tif
L5:   1%I am doing the dirty hack!
L5:   2%I am doing the dirty hack!
Could not map shoreline for this image: 1987-05-23-02-08-21_L5_POYANG2.tif
L5:   2%I am doing the dirty hack!
Could not map shoreline for this image: 1987-06-24-02-09-06_L5_POYANG2.tif
L5:   3%I am doing the dirty hack!
L5:   4%I am doing the dirty hack!
L5:   4%I am doing the dirty hack!
L5:   5%I am doing the dirty hack!
L5:   5%I am doing the dirty hack!
Could not map shoreline for this image: 1988-06-26-02-14-58_L5_POYANG2.tif
L5:   6%I am doing the dirty hack!
Could not map shoreline for this image: 1988-07-28-02-15-04_L5_POYANG2.tif
L5:   6%I am doing the dirty hack!
L5:   6%I am doing the dirty hack!
L5:   7%I am doing the dirty hack!
L5:   7%I am doing the dirty hack!
L5:   7%I am doing the dirty hack!
L5:   8%I am doing the dirty hack!
Could not map shoreline for this image: 1989-

In [ ]:
fig = plt.figure()
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
plt.legend()
mng = plt.get_current_fig_manager()                                         
mng.window.showMaximized()    
fig.set_size_inches([15.76,  8.52])

In [ ]:
transect_settings = {
    'reference_filename': "poyang_refline1.geojson",
    'spacing': 50,
    'section_length': 800,
    'additional inputs': settings
    }

In [ ]:
transects_gdf = SDS_transects.auto_comp_transects(transect_settings)
transects_gdf.plot()

In [ ]:
geojson_file = os.path.join(os.getcwd(), inputs['sitename'], '_transects.geojson')
transects = SDS_tools.transects_from_geojson(geojson_file)